In [1]:
import os
import pandas as pd

In [13]:
from multiprocessing import Pool

In [2]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [3]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

In [4]:
df_revision = pd.read_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]003_assign_labels.p"))
df_revision.head()

,comment,dataset_file,model,note,page_id,page_ns,page_title,timestamp,user_id,user_ip,user_name,action_base,action_digit,action_language,label
0,/* wbsetdescription-add:1|ar */ مؤرخ ألماني,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101383,0,Q98773,2017-06-23 19:37:19+00:00,2437879,,Benseid Seid,wbsetdescription-add,1,ar,DESCRIPTION_ADD
1,"/* wbsetlabel-add:1|sl */ Gerhard Raff, #quick...",wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101383,0,Q98773,2018-01-29 13:38:28+00:00,23475,,Sporti,wbsetlabel-add,1,sl,LABEL_ADD
2,/* wbsetsitelink-add:1|enwiki */ Wolfram Wette,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-01-04 17:08:50+00:00,17848,,Assayer,wbsetsitelink-add,1,enwiki,SITELINK_ADD
3,/* wbsetreference-add:2| */ [[Property:P937]]:...,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-06-06 08:21:58+00:00,38324,,Olaf Kosinsky,wbsetreference-add,2,,REFERENCE_ADD
4,/* wbcreateclaim-create:1| */ [[Property:P1412...,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-06-06 14:59:40+00:00,38324,,Olaf Kosinsky,wbcreateclaim-create,1,,CLAIM_CREATE


In [6]:
grouped_users = df_revision.groupby("user_id")

In [7]:
break_label = cfg.get("preprocessing", "break_label")
break_delta = pd.Timedelta(cfg.get("preprocessing", "break_min_delta"))

In [19]:
# use this implementation for singlecore
users = []
for user_id, revisions in tqdm(grouped_users):
    sequence = []
    last_ts = None
    for i, (_, r) in enumerate(revisions.sort_values("timestamp").iterrows()):
        if i > 0:
            if r['timestamp'] > last_ts + break_delta:
                sequence.append(break_label)
        sequence.append(r['label'])
        last_ts = r['timestamp']
    users.append({"user_id": user_id, "length": len(sequence), "sequence": sequence})

In [8]:
df_user = pd.DataFrame(users)

In [9]:
df_user.head()

,length,sequence,user_id
0,1523,"[CLAIM_CREATE, CLAIM_CREATE, BREAK, DESCRIPTIO...",1
1,8,"[SITELINK_ADD, ENTITY_CREATE, BREAK, MERGE, BR...",1000036
2,7,"[SITELINK_ADD, BREAK, SITELINK_ADD, BREAK, SIT...",1000078
3,8,"[SITELINK_REMOVE, SITELINK_ADD, BREAK, SITELIN...",100008
4,35,"[SITELINK_ADD, SITELINK_ADD, BREAK, SITELINK_A...",100012


### Store data

In [11]:
df_user.to_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_sequences.p"))

In [15]:
all_labels = set(df_revision['label'].unique())
all_labels.add(break_label)
all_labels = sorted(all_labels)
df_label = pd.DataFrame({"label": all_labels})
# df_label

In [16]:
df_label.to_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]004_labels.p"))